#### Incremental Learning- Model Retraining Approach

In [1]:
import pandas as pd

messages = pd.read_csv('smsspamcollection/SMSSpamCollection', sep='\t',
                           names=["label", "message"])

In [2]:
messages.shape

(5572, 2)

In [3]:
from sklearn.model_selection import train_test_split
message_train,message_test=train_test_split(messages)

In [5]:
message_train

,label,message
2428,ham,Do you think i can move &lt;#&gt; in a week
4277,ham,Can you please send me my aunty's number
2539,ham,The monthly amount is not that terrible and yo...
4950,ham,I am in bus on the way to calicut
1019,ham,Nope watching tv at home... Not going out. V b...
...,...,...
4923,ham,Hi Dear Call me its urgnt. I don't know whats ...
3159,ham,Thank u. IT BETTER WORK OUT CAUSE I WILL FEEL ...
5263,ham,Pls speak with me. I wont ask anything other t...
5075,ham,I want to sent &lt;#&gt; mesages today. Thats...


In [6]:
# Convert dataframe to list of tuples
messages_train = message_train.to_records(index=False)
messages_test=message_test.to_records(index=False)

In [7]:
messages_train

rec.array([('ham', 'Do you think i can move  &lt;#&gt;  in a week'),
           ('ham', "Can you please send me my aunty's number"),
           ('ham', 'The monthly amount is not that terrible and you will not pay anything till 6months after finishing school.'),
           ...,
           ('ham', 'Pls speak with me. I wont ask anything other then you friendship.'),
           ('ham', 'I want to sent  &lt;#&gt; mesages today. Thats y. Sorry if i hurts'),
           ('ham', "Hi! You just spoke to MANEESHA V. We'd like to know if you were satisfied with the experience. Reply Toll Free with Yes or No.")],
          dtype=[('label', 'O'), ('message', 'O')])

In [10]:
# Creating the pipeline
# 1st function is creating the TFIDF
# 2nd function is the naive bayes predictor
import math
from creme import compose
from creme import feature_extraction
from creme import naive_bayes
import creme
model = compose.Pipeline(
    ('tokenize', feature_extraction.TFIDF(lowercase=False)),
    ('nb', naive_bayes.MultinomialNB(alpha=1))
)

In [11]:
from creme import metrics
metric=metrics.Accuracy()
# Training the model row by row
for label,sentence in messages_train:
    model = model.fit_one(sentence, label)
    y_pred = model.predict_one(sentence)
    metric = metric.update(label, y_pred)
   

In [12]:
### Training Data Accuracy
metric

Accuracy: 95.84%

In [13]:
### test Data Accuracy
test_metric=metrics.Accuracy()
for label,sentence in messages_test:
    y_pred = model.predict_one(sentence)
    test_metric = metric.update(label, y_pred)

In [14]:
### test Metric
test_metric

Accuracy: 95.89%

In [15]:
model.fit_one("This guy is neutral", "ham")
model.fit_one("Everybody is neutral ", "ham")

Pipeline (
  TFIDF (
    normalize=True
    on=None
    strip_accents=True
    lowercase=False
    preprocessor=None
    tokenizer=<built-in method findall of re.Pattern object at 0x0000029953828030>
    ngram_range=(1, 1)
  ),
  MultinomialNB (
    alpha=1
  )
)